# Introduction

This notebook compares the original NN and the PSO-NN for different datasets.

The dataset can be used via libraries like `sklearn` or `tensorflow`. The datasets used are:
- Iris
- Wine
- Breast Cancer

Comparing the performance of PSO-based optimization with traditional backpropagation
in terms of accuracy and convergence speed.
✓ Comparing behaviour and results in three different datasets your choice. You are
allowed to choose any, but you are expected to reason, explain and compare the results.
✓ Analyzing the influence of various PSO parameters (w, c1, c2, and velocity limits) on
optimization performance.
✓ Identifying strengths and weaknesses of PSO for neural network training, particularly in
terms of handling high-dimensional search spaces.

In [13]:
#Loading all 3 dataset and creating train test splits.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#import iris dataset from sklearn
from sklearn.datasets import load_iris,load_breast_cancer,load_wine


#Loading the dataset from the internet
datasets = [load_iris(),load_breast_cancer(),load_wine()]
dataset_names = ['iris','breast_cancer','wine']
#create train test splits for each dataset and put them in a dictionary
sets = []
for i,dataset in enumerate(datasets):
    datadict ={}
    x_train,x_test,y_train,y_test = train_test_split(dataset.data,dataset.target,test_size=0.2,random_state=42)
    datadict['x_train'] = x_train
    datadict['x_test'] = x_test
    datadict['y_train'] = y_train
    datadict['y_test'] = y_test
    sets.append(datadict)
    
print(sets[0].keys())


dict_keys(['x_train', 'x_test', 'y_train', 'y_test'])


# Perofrmance Comparison

Comparing the performance of PSO-based optimization with traditional backpropagation
in terms of accuracy and convergence speed.

In [9]:
import importlib
import timeit

# Import modules
PSO = importlib.import_module("PSO-NN")
CS = importlib.import_module("classic-NN")
from commonsetup import n_hidden, X_train, X_test, y_train, y_test, n_inputs, n_classes, activation, n_iteration, learning_rate

# Set PSO parameters
par_C1 = 2.0
par_C2 = 2.0
par_W = 0.7
par_SwarmSize = 100
batchsize = 200  # Number of data instances used by the fitness function

# Print settings
print("############ You are using the following settings:")
print("Number of hidden layers:", n_hidden)
print("Activation function:", activation[0])
print("Number of variables to optimize:", (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes)
print("PSO parameters C1:", par_C1, "C2:", par_C2, "W:", par_W, "SwarmSize:", par_SwarmSize, "Iterations:", n_iteration)
print("\n")

nn = PSO.NeuralNetwork(n_inputs, n_hidden, n_classes, activation[0])
# Define function for PSO optimization
def optimize_pso():
    
    pso = PSO.PSOOptimizer(nn, par_C1, par_C2, par_W, par_SwarmSize, n_iteration, batchsize)
    weights = pso.optimize(X_train, y_train)
    return weights

# Measure time for PSO optimization
pso_time = timeit.timeit(optimize_pso, number=1)
print(f"PSO optimization time: {pso_time:.2f} seconds")

# Evaluate PSO-NN accuracy
weights = optimize_pso()
y_pred = nn.predict(weights, X_test)
pso_accuracy = (y_pred == y_test).mean()
print(f"Accuracy PSO-NN: {pso_accuracy:.2f}")

# Define function for backpropagation training
def train_backprop():
    nn = CS.NeuralNetwork(n_inputs, n_hidden, n_classes, activation[0], activation[1])
    nn.train(X_train, y_train, n_iteration, learning_rate)
    return nn

# Measure time for backpropagation training
backprop_time = timeit.timeit(train_backprop, number=1)
print(f"Backpropagation training time: {backprop_time:.2f} seconds")

# Evaluate Classic-NN accuracy
nn = train_backprop()
y_pred = nn.predict(X_test)
backprop_accuracy = (y_pred == y_test).mean()
print(f"Accuracy Classic-NN: {backprop_accuracy:.2f}")

# Compare results
print("\nComparison Results:")
print(f"PSO-NN Accuracy: {pso_accuracy:.2f} | Time: {pso_time:.2f} seconds")
print(f"Classic-NN Accuracy: {backprop_accuracy:.2f} | Time: {backprop_time:.2f} seconds")

############ You are using the following settings:
Number of hidden layers: 10
Activation function: <function sigmoid at 0x0000022B82AC4A60>
Number of variables to optimize: 332
PSO parameters C1: 2.0 C2: 2.0 W: 0.7 SwarmSize: 100 Iterations: 1000


PSO optimization time: 11.37 seconds
Accuracy PSO-NN: 0.99
Epoch 0, Loss: 0.6947
Epoch 100, Loss: 0.6599
Epoch 200, Loss: 0.6597
Epoch 300, Loss: 0.6597
Epoch 400, Loss: 0.6597
Epoch 500, Loss: 0.6597
Epoch 600, Loss: 0.6597
Epoch 700, Loss: 0.6597
Epoch 800, Loss: 0.6597
Epoch 900, Loss: 0.6597
Backpropagation training time: 0.26 seconds
Epoch 0, Loss: 0.6918
Epoch 100, Loss: 0.6504
Epoch 200, Loss: 0.6395
Epoch 300, Loss: 0.6332
Epoch 400, Loss: 0.5562
Epoch 500, Loss: 0.5209
Epoch 600, Loss: 0.5023
Epoch 700, Loss: 0.4380
Epoch 800, Loss: 0.4743
Epoch 900, Loss: 0.3992
Accuracy Classic-NN: 0.91

Comparison Results:
PSO-NN Accuracy: 0.99 | Time: 11.37 seconds
Classic-NN Accuracy: 0.91 | Time: 0.26 seconds


Classical NN is way faster but has a worse accuracy than PSO-NN. PSO-NN is slower but has a better accuracy.